In [ ]:
import random
import time

# Game Configuration
GAME_CONSTANTS = {
    "NUM_PLAYERS": 3,
    "PIECES_PER_PLAYER": 3,
    "BOARD_SIZE": 52,
    "HOME_PATH_SIZE": 6,
    "STARTING_STRATEGY_POINTS": 5,
    "POINTS_PER_CAPTURE": 2,
    "POINTS_PER_HOME": 1,
    "COST_REROLL": 3,
}

PLAYER_CONFIG = {
    "COLORS": ["Red", "Green", "Blue"],
    "START_POSITIONS": {"Red": 0, "Green": 13, "Blue": 26},
    "HOME_ENTRANCE": {"Red": 51, "Green": 12, "Blue": 25},
}

BOARD_CONFIG = {
    "SAFE_ZONES": [0, 8, 13, 21, 26, 34, 39, 47],
    "STATE_HOME": -1,
    "STATE_FINISHED": 999,
}

# Helper Functions
def roll_dice():
    return random.randint(1, 6)

def get_piece_display(player_index, piece_index):
    return f"{PLAYER_CONFIG['COLORS'][player_index][0]}{piece_index + 1}"

# Game State Class
class GameState:
    def __init__(self):
        self.pieces = {
            i: [BOARD_CONFIG["STATE_HOME"]] * GAME_CONSTANTS["PIECES_PER_PLAYER"]
            for i in range(GAME_CONSTANTS["NUM_PLAYERS"])
        }
        self.strategy_points = {
            i: GAME_CONSTANTS["STARTING_STRATEGY_POINTS"]
            for i in range(GAME_CONSTANTS["NUM_PLAYERS"])
        }
        self.current_player_index = 0
        self.dice_roll = 0
        self.turn_phase = "ROLL"
        self.message = ""

    def next_player(self):
        self.current_player_index = (self.current_player_index + 1) % GAME_CONSTANTS["NUM_PLAYERS"]
        self.turn_phase = "ROLL"
        self.message = ""

    def get_player_color(self, player_index=None):
        if player_index is None:
            player_index = self.current_player_index
        return PLAYER_CONFIG["COLORS"][player_index]

    def get_absolute_position(self, player_index, relative_pos):
        if relative_pos in [BOARD_CONFIG["STATE_HOME"], BOARD_CONFIG["STATE_FINISHED"]]:
            return relative_pos
        start_pos = PLAYER_CONFIG["START_POSITIONS"][self.get_player_color(player_index)]
        return (start_pos + relative_pos) % GAME_CONSTANTS["BOARD_SIZE"]

    def calculate_target_position(self, player_index, piece_index, roll):
        current_pos = self.pieces[player_index][piece_index]
        color = self.get_player_color(player_index)
        start_pos = PLAYER_CONFIG["START_POSITIONS"][color]
        home_entry_square = PLAYER_CONFIG["HOME_ENTRANCE"][color]

        if current_pos == BOARD_CONFIG["STATE_HOME"] and roll != 6:
            return None
        if current_pos == BOARD_CONFIG["STATE_FINISHED"]:
            return BOARD_CONFIG["STATE_FINISHED"]

        if current_pos < GAME_CONSTANTS["BOARD_SIZE"]:
            potential_pos = (current_pos + roll if roll != 3 else current_pos - roll) % GAME_CONSTANTS["BOARD_SIZE"]
            if current_pos <= home_entry_square and potential_pos > home_entry_square:
                steps_after_entry = potential_pos - home_entry_square - 1
                if steps_after_entry < GAME_CONSTANTS["HOME_PATH_SIZE"]:
                    home_path_pos = GAME_CONSTANTS["BOARD_SIZE"] + steps_after_entry
                    if home_path_pos == GAME_CONSTANTS["BOARD_SIZE"] + GAME_CONSTANTS["HOME_PATH_SIZE"] - 1:
                        return BOARD_CONFIG["STATE_FINISHED"]
                    if home_path_pos < GAME_CONSTANTS["BOARD_SIZE"] + GAME_CONSTANTS["HOME_PATH_SIZE"]:
                        return home_path_pos
                return None
            return potential_pos % GAME_CONSTANTS["BOARD_SIZE"]

        elif current_pos >= GAME_CONSTANTS["BOARD_SIZE"]:
            steps_already_on_path = current_pos - GAME_CONSTANTS["BOARD_SIZE"]
            potential_home_steps = steps_already_on_path + roll if roll != 3 else steps_already_on_path - roll
            if potential_home_steps == GAME_CONSTANTS["HOME_PATH_SIZE"] - 1:
                return BOARD_CONFIG["STATE_FINISHED"]
            if 0 <= potential_home_steps < GAME_CONSTANTS["HOME_PATH_SIZE"]:
                return GAME_CONSTANTS["BOARD_SIZE"] + potential_home_steps
        return None

    def get_valid_moves(self):
        valid_moves = []
        player_idx = self.current_player_index
        roll = self.dice_roll

        for i in range(GAME_CONSTANTS["PIECES_PER_PLAYER"]):
            current_pos = self.pieces[player_idx][i]
            if current_pos == BOARD_CONFIG["STATE_FINISHED"]:
                continue
            target_pos = self.calculate_target_position(player_idx, i, roll)
            if target_pos is not None:
                can_move = True
                if target_pos != BOARD_CONFIG["STATE_FINISHED"] and target_pos < GAME_CONSTANTS["BOARD_SIZE"]:
                    for j in range(GAME_CONSTANTS["PIECES_PER_PLAYER"]):
                        if i != j and self.pieces[player_idx][j] == target_pos and target_pos not in BOARD_CONFIG["SAFE_ZONES"]:
                            can_move = False
                            break
                if can_move:
                    valid_moves.append((i, i))
        return valid_moves

    def move_piece(self, piece_index):
        player_idx = self.current_player_index
        roll = self.dice_roll
        valid_moves = self.get_valid_moves()

        if piece_index not in [move[1] for move in valid_moves]:
            self.message = "Invalid piece selected."
            return False

        move_to_make = self.calculate_target_position(player_idx, piece_index, roll)
        if move_to_make is None:
            self.message = "Invalid move."
            return False

        old_pos = self.pieces[player_idx][piece_index]
        new_pos = move_to_make
        captured_opponent = False

        if new_pos != BOARD_CONFIG["STATE_FINISHED"] and new_pos < GAME_CONSTANTS["BOARD_SIZE"] and new_pos not in BOARD_CONFIG["SAFE_ZONES"]:
            for opp_idx in range(GAME_CONSTANTS["NUM_PLAYERS"]):
                if opp_idx != player_idx:
                    for opp_piece_idx in range(GAME_CONSTANTS["PIECES_PER_PLAYER"]):
                        if self.pieces[opp_idx][opp_piece_idx] == new_pos:
                            self.pieces[opp_idx][opp_piece_idx] = BOARD_CONFIG["STATE_HOME"]
                            captured_opponent = True
                            self.strategy_points[player_idx] += GAME_CONSTANTS["POINTS_PER_CAPTURE"]
                            self.message = f"Captured {self.get_player_color(opp_idx)}'s piece! (+{GAME_CONSTANTS['POINTS_PER_CAPTURE']} strategy points)"
                            break
                if captured_opponent:
                    break

        self.pieces[player_idx][piece_index] = new_pos
        if not captured_opponent:
            self.message = (
                f"Moved {get_piece_display(player_idx, piece_index)} from "
                f"{old_pos if old_pos != BOARD_CONFIG['STATE_HOME'] else 'Yard'} to "
                f"{new_pos if new_pos != BOARD_CONFIG['STATE_FINISHED'] else 'Home'}."
            )

        if new_pos == BOARD_CONFIG["STATE_FINISHED"]:
            self.strategy_points[player_idx] += GAME_CONSTANTS["POINTS_PER_HOME"]
            self.message += f" Piece reached home! (+{GAME_CONSTANTS['POINTS_PER_HOME']} strategy point)"

        if roll == 6 or captured_opponent or new_pos == BOARD_CONFIG["STATE_FINISHED"]:
            self.message += " Extra turn earned!"
            self.turn_phase = "ROLL"
            self.dice_roll = 0
            return True
        else:
            self.next_player()
            return True

    def check_win_condition(self):
        return all(pos == BOARD_CONFIG["STATE_FINISHED"] for pos in self.pieces[self.current_player_index])

    def apply_reroll(self):
        player_idx = self.current_player_index
        if self.strategy_points[player_idx] >= GAME_CONSTANTS["COST_REROLL"]:
            self.strategy_points[player_idx] -= GAME_CONSTANTS["COST_REROLL"]
            self.dice_roll = roll_dice()
            self.message = f"Paid {GAME_CONSTANTS['COST_REROLL']} points to reroll. New roll: {self.dice_roll}"
            self.turn_phase = "CHOOSE_PIECE"
            return True
        self.message = f"Not enough strategy points to reroll (need {GAME_CONSTANTS['COST_REROLL']}, have {self.strategy_points[player_idx]})."
        return False

# CLI Game Class
class LudoCLI:
    def __init__(self):
        self.game_state = GameState()
        self.running = True

    def display_board(self):
        print("\n=== Ludo Board ===")
        # Collect occupied squares
        occupied_squares = {}
        for player_idx in range(GAME_CONSTANTS["NUM_PLAYERS"]):
            color = PLAYER_CONFIG["COLORS"][player_idx]
            for piece_idx, pos in enumerate(self.game_state.pieces[player_idx]):
                if pos < GAME_CONSTANTS["BOARD_SIZE"] and pos != BOARD_CONFIG["STATE_HOME"]:
                    abs_pos = self.game_state.get_absolute_position(player_idx, pos)
                    piece = get_piece_display(player_idx, piece_idx)
                    if abs_pos not in occupied_squares:
                        occupied_squares[abs_pos] = []
                    occupied_squares[abs_pos].append(piece)
        # Display only occupied squares
        if occupied_squares:
            print("Occupied Squares:")
            for square in sorted(occupied_squares.keys()):
                safe = "*" if square in BOARD_CONFIG["SAFE_ZONES"] else ""
                pieces = "/".join(occupied_squares[square])
                print(f"Square {square:2}{safe}: {pieces}")
        else:
            print("No pieces on the board.")
        # Display home paths
        print("\nHome Paths:")
        for color in PLAYER_CONFIG["COLORS"]:
            home_pieces = []
            for piece_idx, pos in enumerate(self.game_state.pieces[PLAYER_CONFIG["COLORS"].index(color)]):
                if pos >= GAME_CONSTANTS["BOARD_SIZE"] and pos != BOARD_CONFIG["STATE_FINISHED"]:
                    step = pos - GAME_CONSTANTS["BOARD_SIZE"]
                    home_pieces.append(f"{get_piece_display(PLAYER_CONFIG['COLORS'].index(color), piece_idx)} at step {step}")
            print(f"{color}: {', '.join(home_pieces) if home_pieces else 'None'}")

    def display_game_state(self):
        print("\n=== Game State ===")
        print(f"Current Player: {self.game_state.get_player_color()}")
        print(f"Dice Roll: {self.game_state.dice_roll if self.game_state.dice_roll > 0 else 'Not rolled'}")
        print("\nPlayer Status:")
        for player_idx in range(GAME_CONSTANTS["NUM_PLAYERS"]):
            color = PLAYER_CONFIG["COLORS"][player_idx]
            pieces = []
            for piece_idx, pos in enumerate(self.game_state.pieces[player_idx]):
                piece = get_piece_display(player_idx, piece_idx)
                if pos == BOARD_CONFIG["STATE_HOME"]:
                    pieces.append(f"{piece}: In Yard")
                elif pos == BOARD_CONFIG["STATE_FINISHED"]:
                    pieces.append(f"{piece}: Finished")
                elif pos >= GAME_CONSTANTS["BOARD_SIZE"]:
                    step = pos - GAME_CONSTANTS["BOARD_SIZE"]
                    pieces.append(f"{piece}: Home Path step {step}")
                else:
                    abs_pos = self.game_state.get_absolute_position(player_idx, pos)
                    pieces.append(f"{piece}: Board square {pos} (abs {abs_pos})")
            print(f"{color} (Strategy Points: {self.game_state.strategy_points[player_idx]}):")
            for piece in pieces:
                print(f"  {piece}")
        print("\nMessage:", self.game_state.message or "None")

    def handle_input(self):
        if self.game_state.turn_phase == "ROLL":
            action = input(f"\n{self.game_state.get_player_color()}, press Enter to roll dice or type 'quit' to exit: ").strip().lower()
            if action == "quit":
                self.running = False
                return
            self.game_state.dice_roll = roll_dice()
            self.game_state.message = f"Rolled a {self.game_state.dice_roll}."
            self.game_state.turn_phase = "CHOOSE_PIECE"
            valid_moves = self.game_state.get_valid_moves()
            if not valid_moves:
                self.game_state.message += " No valid moves available."
                time.sleep(1)
                self.game_state.next_player()
        elif self.game_state.turn_phase == "CHOOSE_PIECE":
            valid_moves = self.game_state.get_valid_moves()
            print("\nValid moves:")
            for move in valid_moves:
                piece_idx = move[0]
                piece = get_piece_display(self.game_state.current_player_index, piece_idx)
                target_pos = self.game_state.calculate_target_position(self.game_state.current_player_index, piece_idx, self.game_state.dice_roll)
                if target_pos == BOARD_CONFIG["STATE_FINISHED"]:
                    target = "Home"
                elif target_pos >= GAME_CONSTANTS["BOARD_SIZE"]:
                    target = f"Home Path step {target_pos - GAME_CONSTANTS['BOARD_SIZE']}"
                else:
                    target = f"Board square {target_pos}"
                print(f"  {piece_idx + 1}: Move {piece} to {target}")
            print(f"Strategy Points: {self.game_state.strategy_points[self.game_state.current_player_index]} (Reroll costs {GAME_CONSTANTS['COST_REROLL']})")
            action = input("Enter piece number to move, 'reroll' to reroll, or 'skip' to pass: ").strip().lower()
            if action == "quit":
                self.running = False
            elif action == "reroll":
                if self.game_state.apply_reroll():
                    valid_moves = self.game_state.get_valid_moves()
                    if not valid_moves:
                        self.game_state.message += " No valid moves after reroll."
                        time.sleep(1)
                        self.game_state.next_player()
            elif action == "skip":
                self.game_state.message = "Turn skipped."
                self.game_state.next_player()
            elif action.isdigit() and 1 <= int(action) <= GAME_CONSTANTS["PIECES_PER_PLAYER"]:
                piece_idx = int(action) - 1
                if self.game_state.move_piece(piece_idx):
                    if self.game_state.check_win_condition():
                        self.game_state.turn_phase = "GAME_OVER"
                        self.game_state.message = f"{self.game_state.get_player_color()} wins!"
                else:
                    print("Invalid move. Try again.")
            else:
                print("Invalid input. Enter a piece number, 'reroll', or 'skip'.")

    def run(self):
        print("Welcome to Ludo CLI!")
        while self.running:
            self.display_board()
            self.display_game_state()
            if self.game_state.turn_phase == "GAME_OVER":
                print(f"\nGame Over! {self.game_state.message}")
                self.running = False
            else:
                self.handle_input()
            print("\n" + "=" * 50)
        print("Thanks for playing!")

if __name__ == "__main__":
    game = LudoCLI()
    game.run()

Welcome to Ludo CLI!

=== Ludo Board ===
No pieces on the board.

Home Paths:
Red: None
Green: None
Blue: None

=== Game State ===
Current Player: Red
Dice Roll: Not rolled

Player Status:
Red (Strategy Points: 5):
  R1: In Yard
  R2: In Yard
  R3: In Yard
Green (Strategy Points: 5):
  G1: In Yard
  G2: In Yard
  G3: In Yard
Blue (Strategy Points: 5):
  B1: In Yard
  B2: In Yard
  B3: In Yard

Message: None

Red, press Enter to roll dice or type 'quit' to exit: 


=== Ludo Board ===
No pieces on the board.

Home Paths:
Red: None
Green: None
Blue: None

=== Game State ===
Current Player: Green
Dice Roll: 4

Player Status:
Red (Strategy Points: 5):
  R1: In Yard
  R2: In Yard
  R3: In Yard
Green (Strategy Points: 5):
  G1: In Yard
  G2: In Yard
  G3: In Yard
Blue (Strategy Points: 5):
  B1: In Yard
  B2: In Yard
  B3: In Yard

Message: None

Green, press Enter to roll dice or type 'quit' to exit: 


=== Ludo Board ===
No pieces on the board.

Home Paths:
Red: None
Green: None
Blue: None
